In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-13"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8713,2024-07-13,República Dominicana Lnb,21:00,Reales de la Vega,Titanes Del Licey,1.34,3.06,174.5,1.86,1.84,0.0,0.00,0.00,fHOg7YQ7,0.746269,0.326797,0.537634,0.543478,0.073066,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,80,79,1.47,1.43,0.000,0.000,0.552829,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8714,2024-07-13,Europa Eurobasket Sub,14:30,Bélgica U20 F,Islândia U20 F,1.03,10.50,131.5,1.85,1.85,0.0,0.00,0.00,hIaBG9mg,0.970874,0.095238,0.540541,0.540541,0.066112,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.161544,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8715,2024-07-13,Nova Zelândia Nbl,04:30,Otago Nuggets,Franklin Bulls,3.37,1.29,182.5,1.85,1.85,0.0,0.00,0.00,hGC0NnZr,0.296736,0.775194,0.540541,0.540541,0.071930,311.014,178.440727,0.573739,1.2,1.643168,1.369306,535.20,3.646,2.270359,0.622699,-38.0,151.646,100.668558,0.663839,1.2,1.643168,1.369306,113.88,1.568,0.773802,0.493496,3.0,80,78,6.69,1.46,285.986,172.948,0.631237,0.000000,NaN,0.77,0.154,15.389610,0.000000,0.0,0.000000,0.62,0.124,2.338710,0.000000,0.0,0.000000
8716,2024-07-13,México Lnbp,23:30,Panteras,Correcaminos,1.27,3.53,171.5,1.85,1.85,0.0,0.00,0.00,4hVAvGgl,0.787402,0.283286,0.540541,0.540541,0.070688,203.008,90.907783,0.447804,1.2,1.643168,1.369306,88.06,2.906,1.143910,0.393637,-19.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,74,0,1.19,0.00,135.682,0.000,0.665859,0.000000,NaN,-1.28,-0.256,-1.054687,0.571644,0.3,-0.271644,0.00,0.000,inf,0.000000,0.0,0.000000
8717,2024-07-13,Europa Eurobasket Sub,09:00,Grécia U20 F,Suiça U20 F,1.09,6.41,134.5,1.80,1.90,0.0,0.00,0.00,IJNzErTI,0.917431,0.156006,0.555556,0.526316,0.073437,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.003149,0.038222,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8793,2024-07-13,Austrália Nbl1 Central,07:45,Central Districts Lions,South Adelaide Panthers,2.19,1.63,184.5,1.86,1.80,1.5,2.02,1.49,WQ7nK1S3,0.456621,0.613497,0.537634,0.555556,0.070118,247.926,95.645069,0.385781,2.4,1.341641,0.559017,316.44,2.408,0.981565,0.407627,47.0,119.238,27.689391,0.232220,3.0,0.000000,0.000000,117.39,1.516,0.423415,0.279297,64.0,108,91,2.93,1.29,200.720,163.914,0.207319,0.023184,0.213542,1.26,0.252,4.722222,0.000000,0.0,0.000000,-1.36,-0.272,-2.316176,0.000000,0.0,0.000000
8794,2024-07-13,Austrália Nbl1 Central,07:30,Forestville Eagles,West Adelaide Bearcats,1.61,2.22,179.5,1.80,1.86,0.0,0.00,0.00,2i3rLsDc,0.621118,0.450450,0.555556,0.537634,0.071568,119.034,21.829457,0.183388,2.4,1.341641,0.559017,151.96,1.190,0.123491

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
8757,07:00,Austrália Nbl1 Sul,Eltham,Ballarat,1.97,Back Home
8775,19:30,Eua Nba,Boston Celtics,Miami Heat,2.09,Back Home
